In [4]:
import json
import warnings

warnings.filterwarnings(action='ignore')
import requests
import folium
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import time
import numpy as np  # 수치계산, 배열 처리를 위해 import 한다.
import pandas as pd  # 데이터 분석을 위한 전처리를 위해 import 한다.
import matplotlib as mpl  # matplolib에서 음수 데이터의 '-'부호가 깨지는 것을 방지하기위해 import 한다.

mpl.rcParams['axes.unicode_minus'] = False  # matplolib에서 음수 데이터의 '-'부호가 깨지는 것을 방지한다.
import matplotlib.pyplot as plt  # 시각화를 하기 위해 import 한다.

plt.rcParams["font.size"] = 10  # matplolib에서 사용할 글꼴 크기 설정
plt.rcParams["font.family"] = "NanumGothicCoding"  # matplolib에서 사용할 글꼴 설정
import seaborn as sns  # 시각화를 하기 위해 import 한다. matplolib 기반으로 만든 시각화 도구

pd.options.display.float_format = '{:,.5f}'.format
from plotnine import *  # R의 ggplot2 패키지를 사용하는 것 처럼 사용하는 시각화 도구를 import 한다.
import missingno as msno  # 누락값의 분포를 그래프 형태로 보기위해 import 한다.
import folium  # 지도 시각화를 하기 위해 import 한다.
from wordcloud import WordCloud

In [5]:
api_key =
headers = {'x-api-key': api_key}
url = 'https://open-api.bser.io/v1/'

In [6]:
req = requests.get(url + 'user/nickname?query=Anbin', headers=headers) # 닉네임으로 uid 호출
print(req.text)
time.sleep(1)

{"code": 200, "message": "Success", "user": {"userNum": 1072467, "nickname": "Anbin"}}


In [7]:
req2 = requests.get(url + 'user/stats/1072467/13', headers=headers) # uid/season 으로 유저의 해당 시즌 통계 호출 <구하려는 시즌*2-1  (현재시즌 8)>
time.sleep(1)
req2.text

'{"code": 200, "message": "Success", "userStats": [{"seasonId": 13, "userNum": 1072467, "matchingMode": 3, "matchingTeamMode": 2, "mmr": 1355, "nickname": "Anbin", "rank": 9807, "rankSize": 28094, "totalGames": 13, "totalWins": 3, "totalTeamKills": 69, "escapeCount": 0, "rankPercent": 0.35, "averageRank": 3.69, "averageKills": 2.31, "averageAssistants": 2.69, "averageHunts": 18.23, "top1": 0.23, "top2": 0.38, "top3": 0.46, "top5": 0.77, "top7": 0.92, "characterStats": [{"characterCode": 20, "totalGames": 10, "usages": 10, "maxKillings": 6, "top3": 5, "wins": 2, "top3Rate": 0, "averageRank": 4}, {"characterCode": 45, "totalGames": 2, "usages": 2, "maxKillings": 1, "top3": 0, "wins": 0, "top3Rate": 0, "averageRank": 5}, {"characterCode": 21, "totalGames": 1, "usages": 1, "maxKillings": 6, "top3": 1, "wins": 1, "top3Rate": 1, "averageRank": 1}]}, {"seasonId": 13, "userNum": 1072467, "matchingMode": 3, "matchingTeamMode": 3, "mmr": 2581, "nickname": "Anbin", "rank": 604, "rankSize": 26005,

In [8]:
test = requests.get(url + 'user/stats/725291/13', headers=headers).json()
test = json_normalize(test, 'userStats')
test[(test.matchingTeamMode == 3) & (test.matchingMode == 3)]

,seasonId,userNum,matchingMode,matchingTeamMode,mmr,nickname,rank,rankSize,totalGames,totalWins,...,averageRank,averageKills,averageAssistants,averageHunts,top1,top2,top3,top5,top7,characterStats
2,13,725291,3,3,5208,셀쓰녀가취향,1,30457,802,436,...,2.34000,3.71000,3.83000,26.56000,0.54000,0.63000,0.73000,0.93000,1.00000,"[{'characterCode': 43, 'totalGames': 411, 'usa..."


In [9]:
req3 = requests.get(url + 'user/games/1072467', headers=headers) # uid로 최근 90게임 전적 검색
time.sleep(1)
req3.text

'{"code": 200, "message": "Success", "userGames": [{"userNum": 1072467, "nickname": "Anbin", "gameId": 22614964, "seasonId": 0, "matchingMode": 6, "matchingTeamMode": 4, "characterNum": 3, "skinCode": 1003003, "characterLevel": 20, "gameRank": 1, "playerKill": 11, "playerAssistant": 10, "monsterKill": 43, "bestWeapon": 21, "bestWeaponLevel": 20, "masteryLevel": {"21": 20, "16": 6, "19": 6, "204": 6, "102": 6, "103": 6, "104": 6, "201": 6, "202": 20, "101": 6}, "equipment": {"0": 120402, "1": 202508, "2": 201505, "3": 203403, "4": 204410, "5": 705502}, "versionMajor": 75, "versionMinor": 0, "language": "Korean", "skillLevelInfo": {"1003200": 5, "1003300": 3, "1003400": 5, "1003500": 3, "1003100": 2, "3021000": 2}, "skillOrderInfo": {"1": 1003200, "2": 1003300, "3": 1003400, "4": 1003500, "5": 1003200, "6": 1003200, "7": 1003200, "8": 1003200, "9": 1003100, "10": 3021000, "11": 1003100, "12": 1003500, "13": 1003400, "14": 1003400, "15": 1003400, "16": 1003400, "17": 1003500, "18": 302100

In [10]:
top1000 = requests.get(url + 'rank/top/13/3', headers=headers) # 상위 1001명 정보 호출
print(top1000.text)
time.sleep(1)

{"code": 200, "message": "Success", "topRanks": [{"userNum": 725291, "nickname": "셀쓰녀가취향", "rank": 1, "mmr": 5208, "userEmblems": []}, {"userNum": 207458, "nickname": "SOS", "rank": 2, "mmr": 5203, "userEmblems": []}, {"userNum": 576995, "nickname": "왕왕이", "rank": 3, "mmr": 4954, "userEmblems": []}, {"userNum": 542829, "nickname": "갓루디", "rank": 4, "mmr": 4885, "userEmblems": []}, {"userNum": 570749, "nickname": "ジュルル", "rank": 5, "mmr": 4803, "userEmblems": []}, {"userNum": 463952, "nickname": "데굴데굴미숫가루", "rank": 6, "mmr": 4741, "userEmblems": []}, {"userNum": 1553415, "nickname": "적을찾는피5라", "rank": 7, "mmr": 4723, "userEmblems": []}, {"userNum": 360435, "nickname": "게임적응못한사람", "rank": 8, "mmr": 4655, "userEmblems": []}, {"userNum": 429054, "nickname": "リルパ", "rank": 9, "mmr": 4631, "userEmblems": []}, {"userNum": 2854562, "nickname": "민트초코와플", "rank": 10, "mmr": 4614, "userEmblems": []}, {"userNum": 501191, "nickname": "말파", "rank": 11, "mmr": 4602, "userEmblems": []}, {"userNum": 29

In [11]:
asdf = requests.get(url + 'data/VFCredit', headers=headers)
time.sleep(1)
asdf.json()

{'code': 200,
 'message': 'Success',
 'data': [{'code': 1,
   'mode': 7,
   'phase': 0,
   'conditionType': 'PhaseStart',
   'conditionValue': 0,
   'acquireSelf': 15,
   'acquireTeam': 0},
  {'code': 2,
   'mode': 7,
   'phase': -1,
   'conditionType': 'PhaseStart',
   'conditionValue': 0,
   'acquireSelf': 10,
   'acquireTeam': 0},
  {'code': 3,
   'mode': 7,
   'phase': -1,
   'conditionType': 'AcceleratedPhaseStart',
   'conditionValue': 0,
   'acquireSelf': 10,
   'acquireTeam': 0},
  {'code': 4,
   'mode': 1,
   'phase': -1,
   'conditionType': 'KillMutantChicken',
   'conditionValue': 0,
   'acquireSelf': 3,
   'acquireTeam': 0},
  {'code': 5,
   'mode': 1,
   'phase': -1,
   'conditionType': 'KillMutantBat',
   'conditionValue': 0,
   'acquireSelf': 4,
   'acquireTeam': 0},
  {'code': 6,
   'mode': 1,
   'phase': -1,
   'conditionType': 'KillMutantBoar',
   'conditionValue': 0,
   'acquireSelf': 5,
   'acquireTeam': 0},
  {'code': 7,
   'mode': 1,
   'phase': -1,
   'conditionT

In [12]:
req4 = requests.get(url + 'data/hash', headers=headers)
gg = req4.json()
time.sleep(1)
gg

{'code': 200,
 'message': 'Success',
 'data': {'Achievement': 25497290,
  'ActionCost': 3586852712,
  'ActionCost_Light': 2351618591,
  'Area': 993590539,
  'AreaAttributes': 3029790461,
  'AreaSound': 2271343438,
  'BattlePassMission': 3350932525,
  'BattlePassReward': 2212802826,
  'BattlePassRewardItem': 2758469976,
  'BattleZoneReward': 1214396460,
  'BattlefieldArea': 637359149,
  'Booster': 659324469,
  'BotAiModel': 1889052284,
  'BotCraft': 2283823037,
  'BotMastery': 1925639397,
  'BotNickName': 2061427971,
  'BotSkillBuild': 1316321053,
  'BulletCapacity': 2190560277,
  'BulletCapacity_Light': 1060615826,
  'Character': 44653840,
  'CharacterAttributes': 539867005,
  'CharacterExp': 4274863723,
  'CharacterLevelUpStat': 1117649331,
  'CharacterMastery': 3183597377,
  'CharacterModeModifier': 3266860007,
  'CharacterModeModifier_Light': 3169709200,
  'CharacterSkillVideos': 2536553328,
  'CharacterSkin': 1423839808,
  'CharacterState': 2180826605,
  'CharacterStateGroup': 9507

In [13]:
print(type(top1000))

<class 'requests.models.Response'>


In [14]:
top1000_json = top1000.json()
print(type(top1000_json))
top1000_json

<class 'dict'>


{'code': 200,
 'message': 'Success',
 'topRanks': [{'userNum': 725291,
   'nickname': '셀쓰녀가취향',
   'rank': 1,
   'mmr': 5208,
   'userEmblems': []},
  {'userNum': 207458,
   'nickname': 'SOS',
   'rank': 2,
   'mmr': 5203,
   'userEmblems': []},
  {'userNum': 576995,
   'nickname': '왕왕이',
   'rank': 3,
   'mmr': 4954,
   'userEmblems': []},
  {'userNum': 542829,
   'nickname': '갓루디',
   'rank': 4,
   'mmr': 4885,
   'userEmblems': []},
  {'userNum': 570749,
   'nickname': 'ジュルル',
   'rank': 5,
   'mmr': 4803,
   'userEmblems': []},
  {'userNum': 463952,
   'nickname': '데굴데굴미숫가루',
   'rank': 6,
   'mmr': 4741,
   'userEmblems': []},
  {'userNum': 1553415,
   'nickname': '적을찾는피5라',
   'rank': 7,
   'mmr': 4723,
   'userEmblems': []},
  {'userNum': 360435,
   'nickname': '게임적응못한사람',
   'rank': 8,
   'mmr': 4655,
   'userEmblems': []},
  {'userNum': 429054,
   'nickname': 'リルパ',
   'rank': 9,
   'mmr': 4631,
   'userEmblems': []},
  {'userNum': 2854562,
   'nickname': '민트초코와플',
   'rank': 

In [15]:
topRanks = json_normalize(top1000_json, 'topRanks')
topRanks

,userNum,nickname,rank,mmr,userEmblems
0,725291,셀쓰녀가취향,1,5208,[]
1,207458,SOS,2,5203,[]
2,576995,왕왕이,3,4954,[]
3,542829,갓루디,4,4885,[]
4,570749,ジュルル,5,4803,[]
...,...,...,...,...,...
996,1228334,우숙이,997,2556,[]
997,2144698,冷夜冷某人,998,2556,[]
998,1221355,반찬투정하는동기,999,2555,[]
999,3319648,くるみ,1000,2554,[]


In [16]:
print(type(top1000_json))
print(type(topRanks))

<class 'dict'>
<class 'pandas.core.frame.DataFrame'>


In [17]:
ranker = [] # top1000 uid만 별도로 리스트화
for index, i in topRanks.iterrows():
    # print(f'{topRanks.userNum[i]}')
    ranker.append(topRanks.userNum[index])

In [18]:
print(ranker)

[725291, 207458, 576995, 542829, 570749, 463952, 1553415, 360435, 429054, 2854562, 501191, 294507, 1374507, 3515612, 224287, 572087, 478304, 328575, 218512, 1193238, 3181361, 2952493, 866258, 1936623, 604477, 765136, 3668214, 3167172, 268108, 1393502, 103259, 2851871, 3307720, 693091, 3443475, 2482060, 246974, 848720, 379734, 412738, 3709033, 875248, 40020, 1151134, 108708, 1161005, 2952712, 563813, 1160179, 3702021, 702493, 1250073, 1534874, 384505, 1306380, 1466937, 3031115, 666750, 3772418, 228728, 2120261, 70352, 1484744, 3654679, 2355705, 91414, 1279055, 2190193, 2754248, 2762500, 209353, 2277240, 591743, 45029, 293844, 1167767, 767069, 1651844, 2960633, 1212122, 828577, 3560430, 3890017, 223570, 935564, 293551, 3466750, 300279, 112631, 2230466, 760221, 772291, 549148, 690711, 1962748, 276909, 1199780, 3202997, 1173974, 2764600, 658848, 2491651, 1383248, 474998, 116371, 93340, 827895, 656493, 3310706, 2374732, 2527977, 3646394, 2129859, 58060, 115973, 606681, 1106350, 44146, 10995

In [19]:
df = pd.DataFrame()
req = requests.get(url + 'user/stats/1072467/13', headers=headers)
test = (json_normalize(req.json(), 'userStats'))

# df.append(test[(test.matchingMode==3)&(test.matchingTeamMode==3)])

df2 = test[(test.matchingMode == 3) & (test.matchingTeamMode == 2)]
df3 = df.append(df2, ignore_index=True)
df = df.append(df3)
df

,seasonId,userNum,matchingMode,matchingTeamMode,mmr,nickname,rank,rankSize,totalGames,totalWins,...,averageRank,averageKills,averageAssistants,averageHunts,top1,top2,top3,top5,top7,characterStats
0,13,1072467,3,2,1355,Anbin,9807,28094,13,3,...,3.69000,2.31000,2.69000,18.23000,0.23000,0.38000,0.46000,0.77000,0.92000,"[{'characterCode': 20, 'totalGames': 10, 'usag..."


In [20]:
'''     top 1000 유저 통계(스쿼드만) 데이터프레임화
topRanks_df = pd.DataFrame()
for index,uid in topRanks.iterrows():
    req=requests.get(url+f'user/stats/{uid[0]}/13', headers=headers)
    info = (json_normalize(req.json(),'userStats'))
    info_fin = info[(info.matchingMode==3)&(info.matchingTeamMode==3)]
    time.sleep(1)
    # print(f'{topRanks[topRanks.userNum==info_fin.userNum.iloc[0]]["rank"].iloc[0]}등: {info_fin.nickname.iloc[0]} MMR: {info_fin.mmr.iloc[0]}')
    print(f'{index+1}등: {info_fin.nickname.iloc[0]} MMR: {info_fin.mmr.iloc[0]}')

    topRanks_df = pd.concat([topRanks_df,info_fin], ignore_index=True)
topRanks_df
topRanks_df.drop(1000, inplace=True)
topRanks_df.to_csv('out/topRanks_df.csv')
'''
pass

In [21]:
topRanks_df = pd.read_csv('data/topRanks_df - changed.csv')

In [22]:
topRanks_df

,seasonId,userNum,matchingMode,matchingTeamMode,mmr,nickname,rank,rankSize,totalGames,totalWins,...,averageRank,averageKills,averageAssistants,averageHunts,top1,top2,top3,top5,top7,characterStats
0,13,725291,3,3,5208,셀쓰녀가취향,1,30457,802,436,...,2.34000,3.71000,3.83000,26.56000,0.54000,0.63000,0.73000,0.93000,1,"[{'characterCode': 43, 'totalGames': 411, 'usa..."
1,13,207458,3,3,5203,SOS,2,25384,697,369,...,2.31000,4.04000,3.32000,26.75000,0.53000,0.64000,0.74000,0.94000,1,"[{'characterCode': 1, 'totalGames': 510, 'usag..."
2,13,576995,3,3,4954,왕왕이,3,31102,568,279,...,2.44000,2.68000,5.93000,17.77000,0.49000,0.62000,0.71000,0.92000,1,"[{'characterCode': 17, 'totalGames': 567, 'usa..."
3,13,542829,3,3,4885,갓루디,4,30945,844,386,...,2.39000,2.06000,4.53000,16.45000,0.46000,0.61000,0.73000,0.96000,1,"[{'characterCode': 51, 'totalGames': 427, 'usa..."
4,13,570749,3,3,4803,ジュルル,5,33022,449,196,...,2.49000,3.96000,3.55000,25.68000,0.44000,0.58000,0.71000,0.94000,1,"[{'characterCode': 36, 'totalGames': 117, 'usa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,13,448264,3,3,2556,메갈리안,994,30465,99,23,...,3.22000,1.89000,3.11000,11.75000,0.23000,0.38000,0.56000,0.87000,1,"[{'characterCode': 28, 'totalGames': 30, 'usag..."
996,13,1228334,3,3,2556,우숙이,994,29099,127,34,...,3.13000,3.31000,3.28000,17.12000,0.27000,0.42000,0.57000,0.88000,1,"[{'characterCode': 9, 'totalGames': 109, 'usag..."
997,13,2144698,3,3,2556,冷夜冷某人,994,26189,235,51,...,3.40000,0.96000,3.79000,7.71000,0.22000,0.37000,0.51000,0.83000,1,"[{'characterCode': 41, 'totalGames': 135, 'usa..."
998,13,1221355,3,3,2555,반찬투정하는동기,999,26977,345,34,...,3.53000,1.54000,2.22000,24.96000,0.10000,0.28000,0.51000,0.88000,1,"[{'characterCode': 52, 'totalGames': 171, 'usa..."


In [23]:
'''             top 1000 유저 most3 데이터프레임화
realRank=1
virRank=1
j=topRanks_df.mmr[0]
for index,i in topRanks_df.iterrows():
    if(i.mmr==j):
        topRanks_df.loc[index,'rank']=realRank
        virRank+=1
        j=i.mmr
    else:
        realRank=virRank
        topRanks_df.loc[index,'rank']=realRank
        realRank+=1
        virRank+=1
        j=i.mmr
topRanks_df
topRanks_df.to_csv('out/topRanks_df.csv')
'''
pass

In [24]:
print(topRanks_df)

     seasonId  userNum  matchingMode  matchingTeamMode   mmr  nickname  rank  \
0          13   725291             3                 3  5208    셀쓰녀가취향     1   
1          13   207458             3                 3  5203       SOS     2   
2          13   576995             3                 3  4954       왕왕이     3   
3          13   542829             3                 3  4885       갓루디     4   
4          13   570749             3                 3  4803      ジュルル     5   
..        ...      ...           ...               ...   ...       ...   ...   
995        13   448264             3                 3  2556      메갈리안   994   
996        13  1228334             3                 3  2556       우숙이   994   
997        13  2144698             3                 3  2556     冷夜冷某人   994   
998        13  1221355             3                 3  2555  반찬투정하는동기   999   
999        13  3319648             3                 3  2554       くるみ  1000   

     rankSize  totalGames  totalWins  .

In [25]:
'''             캐릭터 코드 => 캐릭터 한글 이름
characterList = requests.get(url + 'data/Character', headers=headers)
characterList = characterList.json()
time.sleep(1)
characterList = json_normalize(characterList, 'data')
characterList = characterList[['code', 'name']]
characterList.to_csv('out/characterList.csv')
'''
pass

In [26]:
characterList = pd.read_csv('data/characterList - changed.csv')
# characterList.set_index(pd.Index(range(1, 63)), inplace=True)
characterList.set_index('code', inplace=True)

In [27]:
characterList

,name
code,
1,재키
2,아야
3,피오라
4,매그너스
5,자히르
...,...
58,헤이즈
59,아이작
60,타지아


In [28]:
topRanks_df

,seasonId,userNum,matchingMode,matchingTeamMode,mmr,nickname,rank,rankSize,totalGames,totalWins,...,averageRank,averageKills,averageAssistants,averageHunts,top1,top2,top3,top5,top7,characterStats
0,13,725291,3,3,5208,셀쓰녀가취향,1,30457,802,436,...,2.34000,3.71000,3.83000,26.56000,0.54000,0.63000,0.73000,0.93000,1,"[{'characterCode': 43, 'totalGames': 411, 'usa..."
1,13,207458,3,3,5203,SOS,2,25384,697,369,...,2.31000,4.04000,3.32000,26.75000,0.53000,0.64000,0.74000,0.94000,1,"[{'characterCode': 1, 'totalGames': 510, 'usag..."
2,13,576995,3,3,4954,왕왕이,3,31102,568,279,...,2.44000,2.68000,5.93000,17.77000,0.49000,0.62000,0.71000,0.92000,1,"[{'characterCode': 17, 'totalGames': 567, 'usa..."
3,13,542829,3,3,4885,갓루디,4,30945,844,386,...,2.39000,2.06000,4.53000,16.45000,0.46000,0.61000,0.73000,0.96000,1,"[{'characterCode': 51, 'totalGames': 427, 'usa..."
4,13,570749,3,3,4803,ジュルル,5,33022,449,196,...,2.49000,3.96000,3.55000,25.68000,0.44000,0.58000,0.71000,0.94000,1,"[{'characterCode': 36, 'totalGames': 117, 'usa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,13,448264,3,3,2556,메갈리안,994,30465,99,23,...,3.22000,1.89000,3.11000,11.75000,0.23000,0.38000,0.56000,0.87000,1,"[{'characterCode': 28, 'totalGames': 30, 'usag..."
996,13,1228334,3,3,2556,우숙이,994,29099,127,34,...,3.13000,3.31000,3.28000,17.12000,0.27000,0.42000,0.57000,0.88000,1,"[{'characterCode': 9, 'totalGames': 109, 'usag..."
997,13,2144698,3,3,2556,冷夜冷某人,994,26189,235,51,...,3.40000,0.96000,3.79000,7.71000,0.22000,0.37000,0.51000,0.83000,1,"[{'characterCode': 41, 'totalGames': 135, 'usa..."
998,13,1221355,3,3,2555,반찬투정하는동기,999,26977,345,34,...,3.53000,1.54000,2.22000,24.96000,0.10000,0.28000,0.51000,0.88000,1,"[{'characterCode': 52, 'totalGames': 171, 'usa..."


In [29]:
test = topRanks_df.characterStats[2].replace("\'", "\"")
test = {'characterStats': json.loads(test)}
print(test['characterStats'][0])
test_json = {'most': test['characterStats'][0]}
print(characterList.name[test['characterStats'][0]['characterCode']])
print(test.values())

{'characterCode': 17, 'totalGames': 567, 'usages': 567, 'maxKillings': 10, 'top3': 406, 'wins': 279, 'top3Rate': 1, 'averageRank': 2}
아드리아나
dict_values([[{'characterCode': 17, 'totalGames': 567, 'usages': 567, 'maxKillings': 10, 'top3': 406, 'wins': 279, 'top3Rate': 1, 'averageRank': 2}, {'characterCode': 36, 'totalGames': 1, 'usages': 1, 'maxKillings': 0, 'top3': 0, 'wins': 0, 'top3Rate': 0, 'averageRank': 5}]])


In [30]:
mostCharacter = pd.DataFrame()
for index, i in topRanks_df.iterrows():
    characterStats = i.characterStats.replace("\'", "\"")  # df 안에 리스트가 '[]' 상태여서 json 변환을 위해 '와 " replace함
    characterStats_json = {'characterStats': json.loads(characterStats)}
    stats = characterStats_json['characterStats']
    try:
        if stats[2] and stats[1] and stats[0]:
            print(f'{topRanks.nickname[index]}')
            print('most1: {}, 플레이 횟수: {}, 승률: {:.3f}'.format(characterList.name[stats[0]['characterCode']],
                                  (stats[0]['totalGames']),
                                  (stats[0]['wins']/stats[0]['totalGames']*100)))
            print('most2: {}, 플레이 횟수: {}, 승률: {:.3f}'.format(characterList.name[stats[1]['characterCode']],
                                                             (stats[1]['totalGames']),
                                                             (stats[1]['wins']/stats[1]['totalGames']*100)))
            print('most3: {}, 플레이 횟수: {}, 승률: {:.3f}'.format(characterList.name[stats[2]['characterCode']],
                                                             (stats[2]['totalGames']),
                                                             (stats[2]['wins']/stats[2]['totalGames']*100)))
    except:
        try:
            print(f'{topRanks.nickname[index]}')
            if stats[1] and stats[0]:
                print('most1: {}, 플레이 횟수: {}, 승률: {:.3f}'.format(characterList.name[stats[0]['characterCode']],
                                                                 (stats[0]['totalGames']),
                                                                 (stats[0]['wins']/stats[0]['totalGames']*100)))
                print('most2: {}, 플레이 횟수: {}, 승률: {:.3f}'.format(characterList.name[stats[1]['characterCode']],
                                                                 (stats[1]['totalGames']),
                                                                 (stats[1]['wins']/stats[1]['totalGames']*100)))
        except:
            if stats[0]:
                print(f'{topRanks.nickname[index]}')
                print('most1: {}, 플레이 횟수: {}, 승률: {:.3f}'.format(characterList.name[stats[0]['characterCode']],
                                                                 (stats[0]['totalGames']),
                                                                 (stats[0]['wins']/stats[0]['totalGames']*100)))
mostCharacter

셀쓰녀가취향
most1: 셀린, 플레이 횟수: 411, 승률: 60.341
most2: 카밀로, 플레이 횟수: 336, 승률: 52.381
most3: 마르티나, 플레이 횟수: 23, 승률: 30.435
SOS
most1: 재키, 플레이 횟수: 510, 승률: 54.706
most2: 마커스, 플레이 횟수: 86, 승률: 60.465
most3: 현우, 플레이 횟수: 53, 승률: 43.396
왕왕이
most1: 아드리아나, 플레이 횟수: 567, 승률: 49.206
most2: 이바, 플레이 횟수: 1, 승률: 0.000
갓루디
most1: 프리야, 플레이 횟수: 427, 승률: 46.838
most2: 리다이린, 플레이 횟수: 398, 승률: 45.477
most3: 띠아, 플레이 횟수: 18, 승률: 27.778
ジュルル
most1: 이바, 플레이 횟수: 117, 승률: 48.718
most2: 재키, 플레이 횟수: 79, 승률: 40.506
most3: 혜진, 플레이 횟수: 59, 승률: 67.797
데굴데굴미숫가루
most1: 에이든, 플레이 횟수: 190, 승률: 51.579
most2: 얀, 플레이 횟수: 44, 승률: 45.455
most3: 실비아, 플레이 횟수: 42, 승률: 52.381
적을찾는피5라
most1: 하트, 플레이 횟수: 249, 승률: 53.012
most2: 혜진, 플레이 횟수: 35, 승률: 42.857
most3: 피오라, 플레이 횟수: 32, 승률: 40.625
게임적응못한사람
most1: 비앙카, 플레이 횟수: 87, 승률: 51.724
most2: 요한, 플레이 횟수: 46, 승률: 50.000
most3: 시셀라, 플레이 횟수: 22, 승률: 59.091
リルパ
most1: 니키, 플레이 횟수: 216, 승률: 49.537
most2: 프리야, 플레이 횟수: 149, 승률: 40.940
most3: 엘레나, 플레이 횟수: 50, 승률: 34.000
민트초코와플
most1: 마이, 플레이 횟수: 180, 승률: 51

""


In [31]:
mostCharacter = pd.DataFrame()
for index, i in topRanks_df.iterrows():
    characterStats = i.characterStats.replace("\'", "\"") # df 안에 리스트가 '[]' 상태여서 json 변환을 위해 '와 " replace함
    characterStats_json = {'characterStats': json.loads(characterStats)}
    stats = characterStats_json['characterStats']
    print(topRanks.nickname[index])
    for j, most in enumerate(stats[:3]):
        print('most{}: {:>5s}, 플레이 횟수: {:4d}, 승률: {:.2f}'.format(j+1,characterList.name[most['characterCode']],
                                                      (most['totalGames']),
                                                      (most['wins']/most['totalGames']*100)))


셀쓰녀가취향
most1:    셀린, 플레이 횟수:  411, 승률: 60.34
most2:   카밀로, 플레이 횟수:  336, 승률: 52.38
most3:  마르티나, 플레이 횟수:   23, 승률: 30.43
SOS
most1:    재키, 플레이 횟수:  510, 승률: 54.71
most2:   마커스, 플레이 횟수:   86, 승률: 60.47
most3:    현우, 플레이 횟수:   53, 승률: 43.40
왕왕이
most1: 아드리아나, 플레이 횟수:  567, 승률: 49.21
most2:    이바, 플레이 횟수:    1, 승률: 0.00
갓루디
most1:   프리야, 플레이 횟수:  427, 승률: 46.84
most2:  리다이린, 플레이 횟수:  398, 승률: 45.48
most3:    띠아, 플레이 횟수:   18, 승률: 27.78
ジュルル
most1:    이바, 플레이 횟수:  117, 승률: 48.72
most2:    재키, 플레이 횟수:   79, 승률: 40.51
most3:    혜진, 플레이 횟수:   59, 승률: 67.80
데굴데굴미숫가루
most1:   에이든, 플레이 횟수:  190, 승률: 51.58
most2:     얀, 플레이 횟수:   44, 승률: 45.45
most3:   실비아, 플레이 횟수:   42, 승률: 52.38
적을찾는피5라
most1:    하트, 플레이 횟수:  249, 승률: 53.01
most2:    혜진, 플레이 횟수:   35, 승률: 42.86
most3:   피오라, 플레이 횟수:   32, 승률: 40.62
게임적응못한사람
most1:   비앙카, 플레이 횟수:   87, 승률: 51.72
most2:    요한, 플레이 횟수:   46, 승률: 50.00
most3:   시셀라, 플레이 횟수:   22, 승률: 59.09
リルパ
most1:    니키, 플레이 횟수:  216, 승률: 49.54
most2:   프리야, 플레이 횟수:  149, 승률: 40.

In [32]:
'''             top 1000 유저 통계(솔로, 듀오, 스쿼드) 데이터프레임화
rankerStats_df = pd.DataFrame()
for i in ranker:
    rankerStats = requests.get(url + f'user/stats/{i}/13', headers=headers)
    rankerStats2 = json_normalize(rankerStats.json(), 'userStats')
    rankerStats_df = pd.concat([rankerStats_df, rankerStats2], ignore_index=True)
    time.sleep(1)
    print(rankerStats2['characterStats'])
rankerStats_df.to_csv('out/rankerStats_df.csv')
print('fin')
'''
pass

In [32]:
req3 = requests.get(url + 'user/games/1072467', headers=headers) # uid로 최근 90게임 전적 검색
time.sleep(1)
req3.text

In [40]:
top1000 = requests.get(url + 'rank/top/15/3', headers=headers) # 상위 1001명 정보 호출
print(top1000.text)
time.sleep(1)

{"code": 200, "message": "Success", "topRanks": [{"userNum": 725291, "nickname": "셀쓰녀가취향", "rank": 1, "mmr": 2640, "userEmblems": []}, {"userNum": 1199780, "nickname": "탐피", "rank": 2, "mmr": 2637, "userEmblems": []}, {"userNum": 576995, "nickname": "왕왕이", "rank": 3, "mmr": 2633, "userEmblems": []}, {"userNum": 570749, "nickname": "ジュルル", "rank": 4, "mmr": 2566, "userEmblems": []}, {"userNum": 412738, "nickname": "댕비니", "rank": 5, "mmr": 2543, "userEmblems": []}, {"userNum": 2120261, "nickname": "태리프", "rank": 6, "mmr": 2530, "userEmblems": []}, {"userNum": 772427, "nickname": "나나의사생팬", "rank": 7, "mmr": 2528, "userEmblems": []}, {"userNum": 478304, "nickname": "ASeuNa", "rank": 8, "mmr": 2512, "userEmblems": []}, {"userNum": 69146, "nickname": "이비스", "rank": 9, "mmr": 2491, "userEmblems": []}, {"userNum": 268108, "nickname": "지미", "rank": 10, "mmr": 2484, "userEmblems": []}, {"userNum": 501191, "nickname": "말파", "rank": 11, "mmr": 2480, "userEmblems": []}, {"userNum": 116371, "nicknam